# Practical 2: Supply and use tables

## Learning goals
- Understand the structure of supply and use tables
- Practice and interpret the industry aggregation/disaggregation in SUT
- Practice how to transform SUTs into IOT

### Instructions
Download the supply and use tables of the Netherlands and place them in a data folder in your working directory

https://www.cbs.nl/en-gb/custom/2020/29/supply-and-use-input-output-and-sector-accounts.


In [71]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
plt.style.use('bmh')

# Import data

CBS_IO = pd.ExcelFile(f"data/Input-Output tables 2015-2019.xlsx", engine="openpyxl")
CBS_use = pd.ExcelFile(f"data/Use tables 2015-2019.xlsx", engine="openpyxl")
CBS_supply = pd.ExcelFile(f"data/Supply tables 2015-2019.xlsx", engine="openpyxl")

# Print the list of sheets that are available in the xlsx file so that you can investigate what options you have
print(f"\nIO tables: ", *CBS_IO.sheet_names, sep='\n')
print(f"\nUse tables: ", *CBS_use.sheet_names, sep='\n')
print(f"\nSupply tables: ", *CBS_supply.sheet_names, sep='\n')



IO tables: 
IO-table 2015 current prices
IO-table 2016 current prices
IO-table 2016 PY prices
IO-table 2017 current prices
IO-table 2017 PY prices
IO-table 2018 current prices
IO-table 2018 PY prices
IO-table 2019 current prices
IO-table 2019 PY prices
Explanation

Use tables: 
Use 2015 current prices
Use 2016 current prices
Use 2016 previous year prices
Use 2017 current prices
Use 2017 previous year prices
Use 2018 current prices
Use 2018 previous year prices
Use 2019 current prices
Use 2019 previous year prices
Explanation

Supply tables: 
Supply 2015 current prices
Supply 2016 current prices
Supply 2016 PY prices
Supply 2017 current prices
Supply 2017 PY prices
Supply 2018 current prices
Supply 2018 PY prices
Supply 2019 current prices
Supply 2019 PY prices
Explanation


In [74]:
# Select the data at current price for the year 2019

# Parse the IO data for 2019 in current prices
IO_2019_cp = CBS_IO.parse("IO-table 2019 current prices", index_col=1, header=1, skipfooter=3)
IO_2019_cp = IO_2019_cp.iloc[1:, 1:]

# Labels sometimes have leading and trailing white spaces which makes it difficult to then slice the dataframe
IO_2019_cp.columns = IO_2019_cp.columns.str.strip()
IO_2019_cp.index = IO_2019_cp.index.str.strip()
# There could also be errors in labels, always check. It won't matter for this exercise but keep in mind for the future. 
# Examples of mistakes are typos, wrong translaction, encoding issues 

# Parse the Use data for 2019 in current prices
use_2019_cp = CBS_use.parse("Use 2019 current prices", index_col=1, header=1, skipfooter=3)
use_2019_cp = use_2019_cp.iloc[1:, 1:]
# eliminate leading and trailing white spaces in labels
use_2019_cp.columns = use_2019_cp.columns.str.strip()
use_2019_cp.index = use_2019_cp.index.str.strip()

# Parse the supply data for 2019 in current prices
supply_2019_cp = CBS_supply.parse("Supply 2019 current prices", index_col=1, header=1, skipfooter=2)
supply_2019_cp = supply_2019_cp.iloc[1:, 1:]
# eliminate leading and trailing white spaces in labels
supply_2019_cp.columns = supply_2019_cp.columns.str.strip()
supply_2019_cp.index = supply_2019_cp.index.str.strip()

# # Print the columns and index to understand how the data is structured
print("index:\n",*supply_2019_cp.index, sep="\n")
# print("\n\ncolumns:\n",*supply_2019_cp.columns, sep="\n")

index:

Crops and planting material
Live animals and animal products
Agricultural and animal husbandry services
Products of forestry, logging and related services
Fish, other fishing products and support services
Coal and lignite
Crude petroleum and natural gas
Metal ores
Other mining and quarrying products
Mining support services
Fish and meat
Processed and preserved fruit and vegetables
Vegetable and animal oils and fats
Dairy products
Grain, starches, bakery products
Other food products
Prepared animal feeds
Beverages
Tobacco products
Textiles
Wearing apparel
Leather and related products
Wood and products of wood and cork
Paper and paper products
Printing and recording services
Coke and refined petroleum products
Chemicals and chemical products
Basic pharmaceutical products and preparations
Rubber and plastics products
Other non-metallic mineral products
Basic metals
Fabricated metal products
Computer, electronic and optical products
Electrical equipment
Machinery and equipment n.e.

## Exercise 1

1. Select two products
2. Can you quantify their total supply, import, and which industries supply them?
3. Can you perform the same for gross fixed capital formation, value added and employment?

In [75]:
# Select two products

products = ["Dairy products", "Chemicals and chemical products"]

In [76]:
# Slice your data
p_1_supply = supply_2019_cp.loc[products[0]]
p_2_supply = supply_2019_cp.loc[products[1]]

# Isolate non zero values 
p_1_supply = p_1_supply[p_1_supply!=0]
p_2_supply = p_2_supply[p_2_supply!=0]

# Print results
print(f"{products[0]}:\n{p_1_supply}\n\n{products[1]}:\n{p_2_supply}")

Dairy products:
Crop, animal production, hunting and related activities         97.0
Manufacture of food products                                 10223.0
Wholesale trade, except of motor vehicles and motorcycles      211.0
Domestic supply (columns 1-81)                               10531.0
Imports of goods                                              4162.0
Supply at basic prices (columns 82–85)                       14693.0
Trade and transport margins                                   3014.0
Taxes on products                                              563.0
Subsidies on products (–)                                        1.0
Total                                                        18269.0
Name: Dairy products, dtype: float64

Chemicals and chemical products:
Crop, animal production, hunting and related activities            1.0
Other mining and quarrying, except petroleum and gas              92.0
Manufacture of food products                                     249.0
Manufactur

In [77]:
# Slice your data
p_1_use = use_2019_cp.loc[products[0]]
p_2_use = use_2019_cp.loc[products[1]]

# Isolate non zero values 
p_1_use = p_1_use[p_1_use!=0]
p_2_use = p_2_use[p_2_use!=0]

# Print results
print(f"{products[0]}:\n{p_1_use}\n\n{products[1]}:\n{p_2_use}")

Dairy products:
Crop, animal production, hunting and related activities              4.0
Manufacture of food products                                      4026.0
Manufacture of chemicals and chemical products                       4.0
Manufacture of pharmaceutical products and preparations            274.0
Wholesale trade, except of motor vehicles and motorcycles           52.0
Accommodation                                                      118.0
Food and beverage service activities                               622.0
Imputed rents owner-occupided dwellings                              9.0
Public administration and defence; compulsory social security       10.0
Education                                                            9.0
Human health activities                                             81.0
Residential care and social work activities                        282.0
Sports, amusement and recreation activities                         11.0
Other personal service activities  

## Exercise 4
Following the instruction presented during the lecture implement the following two transformations methods from SUTs to IOT:
- Industry Technology Assumption in Product-by-Product format
- Industry Technology Assumption in Industry-by-Industry format

We will only perform the transformation for those methods that don't result in negative values

In [246]:
# First prepare your tables for transforamtion by isolating intermediates from primary inputs and final consumption items

# Use table
U = use_2019_cp.loc[:"Other goods n.e.c.", :"Goods and services n.e.c."] # Use table

### !!! Always pay attention whether your "supply" is in really in supply or make format !!! 
V = supply_2019_cp.loc[:"Other goods n.e.c.", :"Goods and services n.e.c."].T # Make table, the transpose of the supply table

# Final Demand
F = use_2019_cp.loc[:"Other goods n.e.c.", "Exports of goods from production":"Consumption residents in rest of the world"].fillna(0) # Final demand

# Primary inputs
W = use_2019_cp.loc["Other taxes on production":"Operating surplus (net)", :"Goods and services n.e.c."]
# Ensure negative values where needed
# Negative items
W.loc["Other subsidies on production (–)"] = W.loc["Other subsidies on production (–)"] * -1

# Collect your total product and industry outputs
q = supply_2019_cp.loc[:"Other goods n.e.c.","Total"]
g = supply_2019_cp.loc["Supply at basic prices",:"Goods and services n.e.c."]

# Check that they match the sums in the use tables
q_ = U @ np.ones(U.shape[1]) + F @ np.ones(F.shape[1])
g_ = U.T @ np.ones(U.shape[0]) + W.T @ np.ones(W.shape[0])

check_q = (1-q_ / q) * 100
check_g = (1-g_ / g) * 100

print(check_g[check_g!=0])
print(check_q[check_q!=0])

Series([], dtype: float64)
Series([], dtype: float64)


### Model B 

Product-by-product

Industry technology assumption

In [256]:
# Technical coefficient matrix

A = np.matmul(U @ np.linalg.inv(np.diag(g)), V @ np.linalg.inv(np.diag(q)))
A.columns = A.index

# Intermediates
Z = A @ np.diag(q)

# Final Demand

Y = F

# Primary inputs
W_b = np.matmul(W @ np.linalg.inv(np.diag(g)), V)

# Total product output
I = np.eye(q.shape[0])
x_out = 0




C:\Users\fdona\AppData\Local\Temp\ipykernel_31752\3639891327.py:3: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  A = np.matmul(U @ np.linalg.inv(np.diag(g)), V @ np.linalg.inv(np.diag(q)))
C:\Users\fdona\AppData\Local\Temp\ipykernel_31752\3639891327.py:14: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc